### Loading data and libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing, model_selection,feature_extraction

In [2]:
from sklearn.model_selection import RandomizedSearchCV

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from nlp_utils.model import train_model

In [6]:
data_folder = 'data/'

In [7]:
## using the cleaned files
train_data = pd.read_csv(data_folder+'train_clean.csv'); print(train_data.shape)
test_data = pd.read_csv(data_folder+'test_clean.csv'); print(test_data.shape)

(7613, 6)
(3263, 5)


In [8]:
train_data.head()

,id,keyword,location,text,target,text_clean
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order cali...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [9]:
train_data[train_data['target'] == 1].sample()['text'].values[0]

'Arnhem Weather - &lt;p&gt;An unrelenting and dangerous heat wave will expand across the South Central United States\x89Û_ http://t.co/yhAqa5WXoK'

In [10]:
# train_data.location.value_counts()

In [11]:
sample_submission = pd.read_csv(data_folder+'sample_submission.csv')
# sample_submission.head()

### Word Embeddings

#### Pre-processing

In [12]:
from tensorflow.keras.preprocessing import text, sequence

In [13]:
## creating a tokenizer
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(train_data['text'])

In [14]:
word_index = tokenizer.word_index

In [15]:
len(word_index)

22700

In [16]:
# convering tweets to sequences and padding them
train_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(train_data['text']), maxlen=70,padding='post')
test_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(test_data['text']), maxlen=70,padding='post')

In [17]:
train_seq_x

array([[ 119, 4633,   24, ...,    0,    0,    0],
       [ 189,   45,  229, ...,    0,    0,    0],
       [  40, 1751, 1620, ...,    0,    0,    0],
       ...,
       [2824, 2401,  709, ...,    0,    0,    0],
       [  78, 1145,   41, ...,    0,    0,    0],
       [   4,  209,   54, ...,    0,    0,    0]], dtype=int32)

In [18]:
train_x, valid_x, train_y,valid_y = train_test_split(train_seq_x, train_data['target'], test_size = 0.15, random_state = 42 )

In [19]:
### loading pretrained word-embeddings
embeddings_index = {}

for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding='utf-8')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

In [20]:
len(embeddings_index.keys())

999995

In [21]:
### token embedding mapping
embedding_matrix = np.zeros((len(word_index)+1, 300))

x = 0
unfound_words = []
for word, i in word_index.items():
    vec = embeddings_index.get(word)
    if vec is not None:
        x+=1
        embedding_matrix[i] = vec
    else:
        unfound_words.append(word)

print("Embeddings found for {} words out of {}".format(x, len(word_index)))

Embeddings found for 12084 words out of 22700


In [22]:
# unfound_words[400:450]

In [23]:
# embedding_matrix[-1]

In [24]:
# embedding_matrix.sum(axis=1)

### LSTM

In [25]:
from tensorflow.keras import layers

In [26]:
from tensorflow.keras import optimizers

In [27]:
from tensorflow.keras import models

In [28]:
def create_lstm_model():
    
    input_layer = layers.Input(shape=(70,)) # input layer
    embedding_layer = layers.Embedding(len(word_index)+1, 300, trainable = False, weights = [embedding_matrix])(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    lstm_layer = layers.LSTM(100 )(embedding_layer)
#     pooling_layer = layers.GlobalMaxPool1D()(lstm_layer)

    output_layer0 = layers.Dense(128, activation = 'relu')(lstm_layer)
#     output_layer0 = layers.BatchNormalization()(output_layer0)
    output_layer0 = layers.Dropout(0.25)(output_layer0)

#     output_layer1 = layers.Dense(64, activation = 'relu')(output_layer0)
# #     output_layer1 = layers.BatchNormalization()(output_layer1)
#     output_layer1 = layers.Dropout(0.25)(output_layer1)
    
#     output_layer2 = layers.Dense(32, activation = 'relu')(output_layer1)
# #     output_layer2 = layers.BatchNormalization()(output_layer2)
#     output_layer2 = layers.Dropout(0.25)(output_layer2)
    
    output_layer3 = layers.Dense(16, activation = 'relu')(output_layer0)
#     output_layer3 = layers.BatchNormalization()(output_layer3)
#     output_layer3 = layers.Dropout(0.25)(output_layer3)
    
    output_layer4 = layers.Dense(1, activation = 'sigmoid')(output_layer3)
    
    model= models.Model(inputs = input_layer, outputs = output_layer4)
    model.compile(optimizer = optimizers.Adam(learning_rate=0.0001) ,loss = "binary_crossentropy")
    
    return model

In [29]:
### training the model

model = create_lstm_model()


In [30]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 70, 300)           6810300   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 70, 300)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 128)               12928     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064  

In [31]:
train_model(model, train_x, train_y, valid_x, valid_y, 
            neural_network = True, epochs = 200,
            test_vectors = test_seq_x, submissions_data = sample_submission, submissions_file_prefix="lstm_submission" 
           )

Epoch 1/200
203/203 [==============================] - 13s 55ms/step - loss: 0.6838 - val_loss: 0.5728
Epoch 2/200
203/203 [==============================] - 9s 47ms/step - loss: 0.5562 - val_loss: 0.4818
Epoch 3/200
203/203 [==============================] - 9s 44ms/step - loss: 0.4858 - val_loss: 0.4376
Epoch 4/200
203/203 [==============================] - 9s 43ms/step - loss: 0.4534 - val_loss: 0.4347
Epoch 5/200
203/203 [==============================] - 9s 44ms/step - loss: 0.4511 - val_loss: 0.4191
Epoch 6/200
203/203 [==============================] - 9s 44ms/step - loss: 0.4493 - val_loss: 0.4096
Epoch 7/200
203/203 [==============================] - 9s 45ms/step - loss: 0.4380 - val_loss: 0.4291
Epoch 8/200
203/203 [==============================] - 10s 47ms/step - loss: 0.4562 - val_loss: 0.4141
Epoch 9/200
203/203 [==============================] - 10s 49ms/step - loss: 0.4204 - val_loss: 0.4103
Epoch 10/200
203/203 [==============================] - 12s 58ms/step - loss: 0

In [32]:
#0.76, 0.85

### BiLSTM

In [31]:
def create_bilstm_model():
    
    input_layer = layers.Input(shape=(70,)) # input layer
    embedding_layer = layers.Embedding(len(word_index)+1, 300, trainable = False, weights = [embedding_matrix])(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    bilstm_layer = layers.Bidirectional(layers.LSTM(100 ))(embedding_layer)
#     pooling_layer = layers.GlobalMaxPool1D()(lstm_layer)

    output_layer0 = layers.Dense(128, activation = 'relu')(bilstm_layer)
#     output_layer0 = layers.BatchNormalization()(output_layer0)
    output_layer0 = layers.Dropout(0.25)(output_layer0)

    output_layer1 = layers.Dense(64, activation = 'relu')(output_layer0)
#     output_layer1 = layers.BatchNormalization()(output_layer1)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    
#     output_layer2 = layers.Dense(32, activation = 'relu')(output_layer1)
# #     output_layer2 = layers.BatchNormalization()(output_layer2)
#     output_layer2 = layers.Dropout(0.25)(output_layer2)
    
    output_layer3 = layers.Dense(16, activation = 'relu')(output_layer1)
#     output_layer3 = layers.BatchNormalization()(output_layer3)
#     output_layer3 = layers.Dropout(0.25)(output_layer3)
    
    output_layer4 = layers.Dense(1, activation = 'sigmoid')(output_layer3)
    
    model= models.Model(inputs = input_layer, outputs = output_layer4)
    model.compile(optimizer = optimizers.Adam(learning_rate=0.00001) ,loss = "binary_crossentropy")
    
    return model

In [32]:
### training the model

model = create_bilstm_model()


In [33]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 70, 300)           6810300   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 70, 300)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               320800    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256

In [34]:
train_model(model, train_x, train_y, valid_x, valid_y, 
            neural_network = True, epochs = 200,
            test_vectors = test_seq_x, submissions_data = sample_submission, submissions_file_prefix="bilstm_submission" 
           )

Epoch 1/200
203/203 [==============================] - 24s 105ms/step - loss: 0.6944 - val_loss: 0.6922
Epoch 2/200
203/203 [==============================] - 19s 93ms/step - loss: 0.6923 - val_loss: 0.6904
Epoch 3/200
203/203 [==============================] - 21s 105ms/step - loss: 0.6902 - val_loss: 0.6882
Epoch 4/200
203/203 [==============================] - 21s 103ms/step - loss: 0.6888 - val_loss: 0.6853
Epoch 5/200
203/203 [==============================] - 27s 134ms/step - loss: 0.6854 - val_loss: 0.6627
Epoch 6/200
203/203 [==============================] - 26s 129ms/step - loss: 0.6423 - val_loss: 0.5822
Epoch 7/200
203/203 [==============================] - 29s 141ms/step - loss: 0.5846 - val_loss: 0.5464
Epoch 8/200
203/203 [==============================] - 26s 128ms/step - loss: 0.5564 - val_loss: 0.5195
Epoch 9/200
203/203 [==============================] - 26s 127ms/step - loss: 0.5320 - val_loss: 0.5004
Epoch 10/200
203/203 [==============================] - 25s 125ms

In [41]:
# 0.8, 0.86

### RCNN

In [61]:
def create_rcnn():
    input_layer = layers.Input(shape = (70,))
    
    embedding_layer = layers.Embedding(len(word_index)+1,300, trainable = False, weights = [embedding_matrix])(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.25)(embedding_layer)
    
    bilstm_layer = layers.Bidirectional(layers.LSTM(100, return_sequences = True))(embedding_layer)
    
    cnn_layer = layers.Convolution1D(100,3, activation = 'relu')(bilstm_layer)
    
    pooling_layer = layers.GlobalMaxPool1D()(cnn_layer)
    
    output_layer0 = layers.Dense(32, activation = 'relu')(pooling_layer)
    output_layer0 = layers.Dropout(0.25)(output_layer0)
    
    output_layer1 = layers.Dense(16, activation = 'relu')(output_layer0)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    
    output_layer = layers.Dense(1, activation = 'sigmoid')(output_layer1)
    
    model = models.Model(inputs = input_layer, outputs = output_layer)
    model.compile(optimizer = optimizers.Adam(lr = 0.0001), loss = 'binary_crossentropy')
    
    return model

In [62]:
model = create_rcnn()

In [63]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 70)]              0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 70, 300)           6810300   
_________________________________________________________________
spatial_dropout1d_9 (Spatial (None, 70, 300)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 70, 200)           320800    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 68, 100)           60100     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 100)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 32)                3232

In [64]:
train_model(model, train_x, train_y, valid_x, valid_y, 
            neural_network = True, epochs = 200,
            test_vectors = test_seq_x, submissions_data = sample_submission, submissions_file_prefix="rcnn_submission" 
           )

Epoch 1/200
203/203 [==============================] - 31s 136ms/step - loss: 0.6796 - val_loss: 0.5605
Epoch 2/200
203/203 [==============================] - 23s 114ms/step - loss: 0.5563 - val_loss: 0.4596
Epoch 3/200
203/203 [==============================] - 21s 104ms/step - loss: 0.4961 - val_loss: 0.4443
Epoch 4/200
203/203 [==============================] - 22s 106ms/step - loss: 0.4791 - val_loss: 0.4288
Epoch 5/200
203/203 [==============================] - 22s 109ms/step - loss: 0.4516 - val_loss: 0.4306
Epoch 6/200
203/203 [==============================] - 22s 111ms/step - loss: 0.4374 - val_loss: 0.4195
Epoch 7/200
203/203 [==============================] - 23s 111ms/step - loss: 0.4449 - val_loss: 0.4192
Epoch 8/200
203/203 [==============================] - 23s 113ms/step - loss: 0.4497 - val_loss: 0.4157
Epoch 9/200
203/203 [==============================] - 23s 114ms/step - loss: 0.4310 - val_loss: 0.4140
Epoch 10/200
203/203 [==============================] - 26s 126m